<a href="https://colab.research.google.com/github/anoukB/tutorial_from_tracking_to_posture_dynamics/blob/main/Part_3_Transition_Matrix_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting the dynamics

## Imports and Installations
This first section makes sure you have all the necessary functions to run this tutorial smoothly.

In [ ]:
#Install necessary environment for display of videos
!pip install -U kora
from kora.drive import upload_public
from IPython.display import HTML

In [ ]:
#Install necessary environments specific to this notebook

!pip install umap-learn
!pip install umap-learn[plot]
!pip install msmtools

In [ ]:
#Clone the repository with all files, images and videos. This will result in a folder called cloned-repo
!git clone -l -s https://github.com/anoukB/tutorial_from_tracking_to_posture_dynamics cloned-repo
%cd cloned-repo
!ls  #Listo of all elements in the repo

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

***STOP HERE :)***

You need to set the directory for all the files that will be used in the tutorial. The folders from GitHub have been imported in your version of the tutorial, so you need to make sure this is correct. The most likely directory would be the one in the example, but in case anything is different in your case, we will let you set it up.

In [ ]:
#Example:
#directory = '/content/cloned-repo/'

directory ='/content/cloned-repo/'

In [ ]:
#Imports for this tutorial
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import sys
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import os
import scipy
import math
import umap.plot
import matplotlib.animation as anim
from sklearn import preprocessing
from sklearn.cluster import KMeans


#Load functions from other files
import sys
path_to_module = directory
sys.path.append(path_to_module)
import clustering_methods as cl
import operator_calculations as op_calc
import delay_embedding_1D as embed
import matplotlib.animation as anim

In [ ]:
#Make a function to display pictures easily
def display_picture(url):
  im = plt.imread(url)

  fig, axs = plt.subplots(ncols=1, nrows=1)
  axs.imshow(im, cmap='gray')
  axs.set_axis_off()

In [ ]:
# Load the file with the Principal Components TS made in previous tutorial
dir_file_storage = directory  # Choose the directory where you put the file
filename = 'file_principal_components_time_series_larva.csv'  #Write down your filename
PC_ts = np.loadtxt(dir_file_storage + filename  , delimiter = ",")

#Make the trajectory matrix with proper parameters found in Part 2
K_opt = 25
M_opt = 5
traj_matrix = embed.trajectory_matrix(PC_ts,K=K_opt-1)

# Extract the singular modes
u,s,v = np.linalg.svd(traj_matrix,full_matrices=False)
u_withVar = np.dot(u, np.diag(s))[:,:M_opt] #Ponderate U with variance


## Introduction

In the last part of this tutorial series, we aim to interpret the state space built in Part 2 by suggesting a model for its dynamics.

 We will make an important assumption: our system behaves as a Markov model. A dynamical system is Markov if it is memory-less. In other words, if we only need the information in the current step to predict the next. It is a strong assumption, but we will verify its validity in the tutorial. The first reason we are allowed to make this assumption is because we found an embedding of the state space that maximizes predictability. By doing so, we recovered enough information to be able to model the system as Markovian, which is why we will work with the underlying hypothesis that with enough states, we accurately model the system as Markov.


Why do we care about Markovianity?

Because a Markov chain is made from fixed states and fixed probabilities to go from one state to the next. These probabilities can be stored in a matrix called a transition matrix. This very interesting structure will prove a useful analysis tool. Here is an example of a Markov chain with three states and its transition matrix.



In [ ]:
display_picture(directory + 'img_transition_matrix_schematics.png')


In this tutorial, we will first build the transition matrix of our state space. We build this matrix because it contains important information about the dynamics of the system.

You might wonder: why make a transition matrix of we already have the behavior space?

In reality, there is value in both. They are simply different apporaches.

With the state space, we worry about how individual trajectories evolve and study their geometry. When we look at the transition matrix, we look at ensembles of trajectories by looking at transition patterns. One very interesting thing we can (and will) do with the transition matrix is clustering with its eigenvectors. The matrix contains naturally ordered longer-lived dynamics in its eigenvectors, which have a chance of being interpretable.


Computationally, the first step is to identify topological states with k-means clustering, like we did in Part 2. We will then choose a transition time for the transition matrix. Finally, we will decompose the matrix into its eigenvectors and try to interpret them relative to our state space.


 In more formal words, the transition matrix will act as the approximation of the Perron-Froebenius operator that transfer the probaility densities in time and we will apply transfer operators to learn about coarse-grained descriptions. The theoretical basis for this part the the tutorial is explained in [1].

## Calculate optimal N

A Markov chain has states and goes from one to the other with probability $P_ij$. The first step is to identify the states with k-means clustering and entropy rate.

In [ ]:
n_seed_range=np.arange(1,1500,100) #Number of partitions to examine
n_samples = 3  #Number of random samples to make

#Calculate the optimal number of clusters for the embeded space that maximises entropy
# with optimal K and M previously established

h_seeds=[]

for n_seeds in n_seed_range:
    h_samples=[]
    for idx in range(n_samples):
        labels=cl.kmeans_knn_partition(traj_matrix,n_seeds)
        h = op_calc.get_entropy(labels)
        h_samples.append(h)
    h_seeds.append(h_samples)
    print('N =',n_seeds,"Entropy = ",h_samples)

In [ ]:
#Plot the entropy in function of partition number to evaluate the right number of partitions to use in clustering
plt.figure(figsize=(10,6))
mean = np.mean(h_seeds, axis = 1)
std = np.std(h_seeds, axis = 1)
plt.errorbar(n_seed_range,mean,yerr = [ std,std ],capsize=4,marker='o',ms=5)
plt.xlabel('$N$ (partitions)')
plt.ylabel('$h$ (nats/symbol)')
plt.show()

In [ ]:
N_opt = 250  # Optimal number of partitions, will depend on your own graph

## Choose a transition time

The probability of going from one state to the next depends on their topology and the time step we look at. We covered the state's shapes in the last section. In this section we find a time step, or transition time $\tau$, where the system is Markov. A purely Markovian system would be independent of $\tau$. Unfortunately, in real life, when we work with incomplete information, we need to choose it wisely. To do so, we will look at how two metrics evolve with $\tau$: the eigenvalues of the transition matrix and the implied time scale. The implied time scale is the implied relaxation time associated with each eigenvalue, following this equation:
$$
t^{imp}_i (\tau) = \frac{-\tau}{\log\lambda_i(\tau)}
$$

For small $\tau$, the system does not have enough time to leave the current partition, and the eigenvalues will be close to unity. As $\tau$ increases, we leave this transient state. Eventually, for very large $\tau$, the system will reach its mixing state, the eigenvalues will collapse, and the implied time scale will increase linearly. We aim to choose $\tau$ in the intermediate state, after the initial fast dynamics, and the mixed state.  

In [ ]:
range_tau = np.arange(1,100)  #Range of transition times to test
nmodes = 3  #Number of eigenmodes to keep
n_samples = 3  #Number of random samples to make for averaging
dt = 5  #Sampling rate, in frames/second



#Arrays to store all relevant quantities for plotting later
eigvals_mean_arr = np.zeros((n_samples, range_tau.shape[0], nmodes))
eigvals_array = np.zeros((range_tau.shape[0], nmodes))

t_imp_mean_arr = np.zeros((n_samples, range_tau.shape[0], nmodes))
t_imp_array = np.zeros((range_tau.shape[0], nmodes))

for i in range(n_samples):
    print("Bootstrap sample #", i)
    # Make a different version of labels for each bootstrapped sample
    labels_original = ma.array(cl.kmeans_knn_partition(traj_matrix,N_opt),dtype=int)


    for tau in range(0, range_tau.shape[0]):
        print("Transition time: ", tau)
        P = op_calc.transition_matrix(labels_original,tau + 1)  #Make the transition matrix
        R = op_calc.get_reversible_transition_matrix(P)  #Reversibilize the transision matrix
        new_matrix = R.toarray()  #Make the TM into an array
        eigvals, eigvecs = op_calc.sorted_spectrum(new_matrix,k=nmodes)  #Calculate its eigenvalies
        eigvals_array[tau, :] = eigvals.real  #Keep only the real part
        t_imp_array[tau, :] = -(range_tau[tau])/np.log(eigvals.real)  #Calculate t_implied
        t_imp_array[tau, :] = t_imp_array[tau, :]/dt  #Divide it by the sampling rate


    #Store for averaging
    eigvals_mean_arr[i, :, :] = eigvals_array
    t_imp_mean_arr[i, :, :] = t_imp_array

#Calculate means and stp for each value
eigvals_mean = np.mean(eigvals_mean_arr, axis = 0)
eigvals_std = np.std(eigvals_mean_arr, axis = 0)

t_imp_mean = np.mean(t_imp_mean_arr, axis = 0)
t_imp_std = np.std(t_imp_mean_arr, axis = 0)


In [ ]:
#Impose colors of your choice for each mode you want to keep
colors = ["k", "g", "y", "b", "r", "pink"]


for mode in range(1, nmodes):
    plt.semilogx(range_tau,eigvals_mean[:,mode], c = colors[mode], label = '$\lambda$' + str(mode))
    plt.fill_between(range_tau,eigvals_mean[:,mode] - eigvals_std[:,mode],eigvals_mean[:,mode] + eigvals_std[:,mode],alpha=.2, color = colors[mode])

plt.xlabel(r'$\tau (frames)$',fontsize=15)
plt.ylabel('$\lambda$',fontsize=15)
plt.legend()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()


for mode in range(1, nmodes):
    plt.plot(range_tau,t_imp_mean[:,mode], c = colors[mode], label = '$\lambda$' + str(mode))
    plt.fill_between(range_tau,t_imp_mean[:,mode] - t_imp_std[:,mode],t_imp_mean[:,mode] + t_imp_std[:,mode],alpha=.2, color = colors[mode])

plt.xlabel(r'$\tau (frames)$',fontsize=15)
plt.ylabel('$t_{imp}$ (s)',fontsize=15)
plt.legend()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()



From these graphs, we choose an $\tau$ after the initial transient, which occurs between 0 and 20 frames, and the mixing state that seem to start around 60 frames. We will choose $\tau = 25$ frames (5 seconds).

In a true Markov system, the transition matrix at time $\tau$ is equal to the initial transition matrix to the $\tau$ power. This is a version of the Kolmogorov-Chapman identity.

$$ T(\tau) = T_0^\tau $$

We can compare the current eigenvalues with the ones we would obtain with a purely Markov system to give us intuition about how Markov our system is.

In [ ]:
range_tau = np.arange(1,100)  #Range of transition times to test
nmodes = 3  #Number of eigenmodes to keep
n_samples = 3  #Number of random samples to make for averaging
dt = 5  #Sampling rate, in frames/second



#Arrays to store all relevant quantities for plotting later

eigvals_mean_arr_markov = np.zeros((n_samples, range_tau.shape[0], nmodes))
eigvals_array_markov = np.zeros((range_tau.shape[0], nmodes))

for i in range(n_samples):
    print("Bootstrap sample #", i)
    # Make a different version of labels for each bootstrapped sample
    labels_original_markov = ma.array(cl.kmeans_knn_partition(traj_matrix,N_opt),dtype=int)
    P = op_calc.transition_matrix(labels_original_markov, 1)  #Make the transition matrix with tau = 1
    R = op_calc.get_reversible_transition_matrix(P)  #Reversibilize the transision matrix
    initial_matrix_markov = R.toarray()  #Make the TM into an array
    new_matrix_markov = initial_matrix_markov
    eigvals, eigvecs = op_calc.sorted_spectrum(initial_matrix_markov, k=nmodes)
    eigvals_array_markov[0, :] = eigvals.real

    for tau in range(0, range_tau.shape[0]):
        print("Transition time: ", tau)
        new_matrix_markov = np.matmul(new_matrix_markov, initial_matrix_markov)
        eigvals, eigvecs = op_calc.sorted_spectrum(new_matrix_markov,k=nmodes)
        eigvals_array_markov[tau, :] = eigvals.real




    #Store for averaging
    eigvals_mean_arr_markov[i, :, :] = eigvals_array_markov


#Calculate means and stp for each value
eigvals_mean_markov = np.mean(eigvals_mean_arr_markov, axis = 0)
eigvals_std_markov = np.std(eigvals_mean_arr_markov, axis = 0)



In [ ]:
for mode in range(1, nmodes):
    plt.semilogx(range_tau,eigvals_mean[:,mode], c = colors[mode], label = '$\lambda$' + str(mode))
    plt.fill_between(range_tau,eigvals_mean[:,mode] - eigvals_std[:,mode],eigvals_mean[:,mode] + eigvals_std[:,mode],alpha=.2, color = colors[mode])

    plt.semilogx(range_tau,eigvals_mean_markov[:,mode], "--", c = colors[mode], label = ' Markov $ \lambda$' + str(mode))
    plt.fill_between(range_tau,eigvals_mean_markov[:,mode] - eigvals_std_markov[:,mode],eigvals_mean[:,mode] + eigvals_std_markov[:,mode],alpha=.2, color = colors[mode])

plt.xlabel(r'$\tau (frames)$',fontsize=15)
plt.ylabel('$\lambda$',fontsize=15)
plt.legend()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()


The Markov model (dotted line) is very close to the experimental data, which confirms that our assumption is reasonable. We can now make the final transition matrix we will work with for the rest of the tutorial.

## Make the transition matrix

We now have our states ($N$ states) and our transition time $\tau$. We are ready to build the transition matrix that describes how the bending of the larva evolves in time.

In [ ]:
# Final transition matrix with the chosen parameters
labels = ma.array(cl.kmeans_knn_partition(traj_matrix,N_opt),dtype=int)
tau = 25

#Correction of labels excluding states that have not been included in the TM
lcs, P= op_calc.transition_matrix(labels,tau, return_connected = True)
R = op_calc.get_reversible_transition_matrix(P)
transition_matrix_ = R.toarray()
final_labels = op_calc.get_connected_labels(labels,lcs)  #Makes sure only the labels visited in the TM are included


## Analyse the matrix to get longer-lived dynamics

There is something very convenient about the transition matrix: the first eigenvectors describe the longer-lived, or slowest dynamics. We can call these metastable states, which are regions that the system visits often. We can use these eigenvectors to cluster the state space and learn something about it. There is substantial theory beyond the scope of this tutorial behind this statement. If you are interested to understand the underlying assumptions made to do this, here are some useful references [1], [2], [3].

 In this last section, we will analyze this transition matrix by looking at its first eigenmode. First, we plot how the eigenvalues are distributed.

In [ ]:
#Extract and plot the eigenmodes
eigvals,eigvecs = op_calc.sorted_spectrum(transition_matrix_,k = 20)

plt.plot(eigvals.real[1:],"o")
plt.xlabel("Mode", fontsize = 15)
plt.ylabel("Eigenvalue", fontsize = 15)
plt.ylabel
plt.show()

We could analyze each mode individually. Here, we will only look at the first one as an example.

In [ ]:
#Choose which eigenvector you want to look at
mode = 0   #Mode 0 will lead to the first eigenvector
current_eigenvec =eigvecs [final_labels,mode].real  #Look at the current eigenvector with the connected labels

A good first step is to look at how the eigenvector evolves with time.

In [ ]:
#Look at the time series
plt.plot(current_eigenvec)
plt.xlabel("Time (frames)", fontsize = 15)
plt.ylabel(r'$\phi_{1}$', fontsize = 15)
plt.show()

We notice that this vector has a higher amplitude during the first 1000 frames, then stabilizes. It seems to partition two sections of the time series. There is then some dynamics that occur only in the first 1000 frames. If we could look at the full video, we would get an intuitive interpretation of what happens. Indeed, in the first 1000 frames, the larva does some type of short scale exploration. It turns very frequently with very small forward crawling bouts in between. Then, it picks a direction, and starts crawling without turning at all for the rest of the video. This is most likely what the transition matrix's first eigenvector is picking up.


 With a UMAP embedding, we see how the eigenvector partitions the whole space by colouring the time points with the corresponding amplitude of the vector.

In [ ]:
# We normalize the vector
color_abs = np.max(np.abs(current_eigenvec))
scaler = preprocessing.MinMaxScaler(feature_range=(-color_abs, color_abs))
list = current_eigenvec.reshape(-1,1)
normalized_eigenvec = scaler.fit_transform(list)


In [ ]:
#Plot UMAP
reducer = umap.UMAP(n_neighbors=200,min_dist=0.1)
embedding = reducer.fit_transform(u_withVar)
plt.scatter(embedding[:,0], embedding[:, 1],c = normalized_eigenvec, cmap = "coolwarm", s = 0.8)
plt.colorbar()
plt.xticks([])
plt.yticks([])
plt.show()


This plot gives intuition about the structure of the state space itself. The areas with the highest vector amplitudes are all the points from the beginning of the time series (turning), whereas the rest form a conical shape (forward crawling).

To get further intuition about this first eigenvector, we related its amplitude with various parameters, such as the angle of the anterior body of the larva relative displacement, turning speed, etc. We will not give the codes here as they do not contribute to the method, but in the following plot we show first SVD mode against the anterior angle, colored by the amplitude of the first eigenvector.

In [ ]:
display_picture(directory + "img_larva_svd0_vs_ant_angle_vs_phi1.png")

From this plot, we can see that

 1- the first SVD mode is correlated to the anterior angle.

  2- The eigenvector is low (or slightly positive) when the angle is close to 0 (forward crawling, or forward crawling with slight head bend), whereas it is high whenever the angle is important.
  
  This confirms our intuition that the first eigenvector partitions the short-scale exploration with a lot of turning from the long-range forward crawling bouts.

With similar analyses, we could show that the second eigenvector splits right and left turns. As we move further down the eigenvectors, they will describe faster dynamics.

We leave the rest of the analysis to you with the larva data, or your own. We hope this has been interesting at least, maybe useful. Good luck, have fun, and do not hesitate to contact me for questions (see README file)!


$\textit{Disclaimer}$: In this process, we reversibilize the transition matrix to ensure optimal clustering (see [1,2]). This might lead to issues in highlt cyclical behaviors anf prevent interpretability in the Markov test, and even in the eigenvectors themselves.

References

[1] Costa, C. A., et al., "Maximally predictive states: From partial observations to long timescales", Chaos, 2023.

[2] G. Froyland and M. Dellnitz, “Detecting and locating near-optimal almost-invariant sets and cycles,” SIAM J. Sci. Comput. 2003.

[3] G. Froyland, G. A. Gottwald, and A. Hammerlindl, “A computational method to extract macroscopic variables and their dynamics in multiscale systems,” 2014.